<a href="https://colab.research.google.com/github/bray2020/Data-Science-Portfolio-NLP-Works/blob/main/NLP_Topic_Modelling_(LDA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic Modelling:
- Creating Clustering out of the words/text data. Example dividing the areticles or news into different groups/segments
- As we don't know what the topics are stating, so we will sample the topic first and see what the samples are stating
- two types: LDA, LDIA

In [ ]:
import nlpia
from nlpia.data.loaders import get_data

In [ ]:
import pandas as pd
from nlpia.data.loaders import get_data
pd.options.display.width = 120
sms = get_data('sms-spam')
sms.head()

,spam,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
index = ['sms {} {}'.format(i,'|'*j) for (i,j) in zip(range(len(sms)), sms.spam)]

In [ ]:
sms.head()

,spam,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
sms.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4837 entries, 0 to 4836
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   spam    4837 non-null   int64 
 1   text    4837 non-null   object
dtypes: int64(1), object(1)
memory usage: 113.4+ KB


In [ ]:
sms['spam'].value_counts()

0    4199
1     638
Name: spam, dtype: int64

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import casual_tokenize
tfidf_model = TfidfVectorizer(tokenizer = casual_tokenize)
tfidf_docs = tfidf_model.fit_transform(raw_documents=sms.text).toarray()

In [ ]:
tfidf_docs[:2]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
tfidf_docs.shape

(4837, 9232)

In [ ]:
print(sms.spam.sum())

638


In [ ]:
#create mask for clustering of spam and not spam data
mask = sms.spam.astype(bool).values

#for spam data centriod
spam_centroid = tfidf_docs[mask].mean(axis=0)

#for not spam data centroid
ham_centroid = tfidf_docs[~mask].mean(axis=0)

In [ ]:
spamminess_score = tfidf_docs.dot(spam_centroid - ham_centroid)
print(spamminess_score.round(2))

[-0.01 -0.02  0.04 ... -0.01 -0.    0.  ]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
sms['lda_score'] = MinMaxScaler().fit_transform(spamminess_score.reshape(-1,1))

In [ ]:
#using the above lda_score classify the data:
sms['lda_predict'] = (sms.lda_score > 0.5).astype('int')

#display classified data:
sms['spam lda_predict lda_score'.split()].round(2).head(6)

,spam,lda_predict,lda_score
0,0,0,0.23
1,0,0,0.18
2,1,1,0.72
3,0,0,0.18
4,0,0,0.29
5,1,1,0.55
